
## Классификация изображений из датасета [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html)

Иванчин Кирилл


## Задание:

Реализовать алгоримтм позволяющий классифицировать изображения из датасета CIFAR-10 и оценить качество его работы.

Требования:

1. Можете придумать свой алгоритм, загрузить готовую модель или использовать собственную архитектуру.

2. Не используйте предобученные модели.

3. Выберите способ оценки качества предсказаний модели. Обоснуйте его.

4. Проведите обучение. Продемонстрируйте умение использовать соответствующие инструменты.

5. Оцените полученный результат.

*Не используйте инструменты принцип работы которых вам непонятен.

### Данные

In [29]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
from IPython.display import clear_output
from torch import nn, optim
from torchvision import transforms
from tqdm.auto import tqdm
from torchvision import models, datasets
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

warnings.filterwarnings("ignore")
%matplotlib inline

In [30]:
transform = transforms.ToTensor()

# загрузка тренировочной части
cifar_train = datasets.CIFAR10(
    root="./cifar", train=True, download=True, transform=transform
)

# загрузка валидационной части
cifar_val = datasets.CIFAR10(
    root="./cifar", train=False, download=True, transform=transform
)

# создание загрузчика данных для тренировочного набора
train_dataloader = torch.utils.data.DataLoader(
    cifar_train, batch_size=32, shuffle=True, num_workers=4
)

# создание загрузчика данных для валидацонного набора
val_dataloader = torch.utils.data.DataLoader(
    cifar_val, batch_size=32, shuffle=False, num_workers=4
)


Files already downloaded and verified
Files already downloaded and verified


In [31]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Модель

In [32]:
class ModelBaseline(nn.Module):
    def __init__(self, activation=nn.ReLU):
        super().__init__()

        # полносвязный слой для работы с входными данными (32x32x3 = 3072)
        self.fc1 = nn.Linear(32*32*3, 512)

        # функция активации
        self.relu1 = nn.LeakyReLU()

        # второй полносвязный слой
        self.fc2 = nn.Linear(512, 128)

        # функция активации
        self.relu2 = nn.LeakyReLU()

        # последний полносвязный слой с 10 выходами (10 классов)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        # 32x32x3 изображений -> одномерный вектор 3072.
        x = nn.Flatten()(x)

        # полносвязные слои и активации
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))

        # третий полносвязный слой
        x = self.fc3(x)

        return x

### Обучение

In [33]:
def train(model, criterion, optimizer, train_dataloader, val_dataloader, n_epochs=6):
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []

    for epoch in range(n_epochs):
        train_epoch_loss, train_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.train()  # режим тренировки

        for imgs, labels in tqdm(train_dataloader, desc=f"Training, epoch {epoch+1}", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)

            # прямой проход
            y_pred = model(imgs)
            # вычисление потерь
            loss = criterion(y_pred, labels)
            # обратное распространение ошибки
            loss.backward()
            # обновление параметров модели на основе градиентов
            optimizer.step()
            # сброс градиентов до нуля, чтобы подготовиться к следующему шагу
            optimizer.zero_grad()

            # логирование потерь для текущей эпохи и всех эпох
            train_epoch_loss = torch.cat(
                (train_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))  # нормализация потерь по батча
            )
            train_loss_log.append(loss.cpu().data / labels.cpu().size(0))  # логирование потерь для всей тренировки

            # логирование точности для текущей эпохи и всех эпох
            pred_classes = torch.argmax(y_pred.cpu(), dim=-1)  # получение предсказанных классов
            train_epoch_true_hits = torch.cat(
                (
                    train_epoch_true_hits,
                    (pred_classes == labels.cpu()).sum().unsqueeze(0),  # подсчёт верных предсказаний
                )
            )
            train_acc_log.append(
                (pred_classes == labels.cpu()).sum() / labels.cpu().shape[0]  # точность для каждого батча
            )

        # валидация на текущей эпохе
        val_epoch_loss, val_epoch_true_hits = torch.empty(0), torch.empty(0)
        model.eval()
        # отключение вычисления градиентов
        with torch.no_grad():
            for imgs, labels in tqdm(val_dataloader, desc=f"Validating, epoch {epoch+1}", leave=False):
                imgs, labels = imgs.to(device), labels.to(device)

                # прямой проход на валидационных данных
                y_pred = model(imgs)
                # вычисление потерь
                loss = criterion(y_pred, labels)
                val_epoch_loss = torch.cat(
                    (val_epoch_loss, loss.cpu().unsqueeze(0) / labels.cpu().size(0))
                )

                # получение предсказанных классов и логирование правильных предсказаний
                pred_classes = torch.argmax(y_pred.cpu(), dim=-1)
                val_epoch_true_hits = torch.cat(
                    (
                        val_epoch_true_hits,
                        (pred_classes == labels.cpu()).sum().unsqueeze(0),  # подсчёт верных предсказаний
                    )
                )

        # логирование потерь и точности на валидационных данных
        val_loss_log.append(val_epoch_loss.mean())  # средние потери за эпоху
        val_acc_log.append(
            val_epoch_true_hits.sum() / val_epoch_true_hits.size(0) / val_dataloader.batch_size  # точность на валидации
        )
        clear_output()

        # вывод потер и точности для текущих выборок за текущую эпоху
        print("Train loss:", train_epoch_loss.mean().item())
        print(
            "Train acc:",
            (
                train_epoch_true_hits.sum() / train_epoch_true_hits.size(0) / train_dataloader.batch_size
            ).item(),
        )
        print("Val loss:", val_epoch_loss.mean().item())
        print(
            "Val acc:",
            (
                val_epoch_true_hits.sum() / val_epoch_true_hits.size(0) / val_dataloader.batch_size
            ).item(),
        )


In [34]:
model_cifar10 = ModelBaseline().to(device)
criterion = nn.CrossEntropyLoss() # функция потерь
optimizer = optim.SGD(model_cifar10.parameters(), lr=0.001, momentum=0.9) # оптмизатор для град. спуска

train(model_cifar10, criterion, optimizer, train_dataloader, val_dataloader)

Train loss: 0.04807725176215172
Train acc: 0.45503437519073486
Val loss: 0.047521501779556274
Val acc: 0.4628594219684601


### Оценка результата

In [35]:
def valid(model, criterion, dataloader):
    model.eval()
    val_loss = 0.0
    all_labels, all_predictions = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(dataloader, desc="Validating", leave=False):
            imgs, labels = imgs.to(device), labels.to(device)
            y_pred = model(imgs)

            pred_classes = torch.argmax(y_pred, dim=-1)
            all_predictions.extend(pred_classes.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_labels = np.array(all_labels)
    all_predictions = np.array(all_predictions)

    # вычисление метрик
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    accuracy = accuracy_score(all_labels, all_predictions)

    # вывод
    print(f"Accuracy: {accuracy:.3f}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 Score: {f1:.3f}")


In [36]:
valid(model_cifar10, criterion, val_dataloader)

Validating:   0%|          | 0/313 [00:00<?, ?it/s]

Accuracy: 0.464
Precision: 0.472
Recall: 0.464
F1 Score: 0.457


## Вывод

Качество модели получилось достаточно средним. Модель даёт верный ответ чуть меньше чем в половине случаев. По прошлой ячейке можно заметить этот показатель в числовом формате:
*   Точность(Accuracy) = 0.46, что говорит о том что, модель верно определила класс чуть более 46% изображений
*   Точность(Precision) = 0.47
*   Полнота(Recall) = 0.46, это означает, что модель находит чуть больше 46% всех объектов каждого класса.
*   F1-мер = 0.46. Низкое значение говорит о том, что и точность, и полнота модели достаточно посредственны.

Думаю, что для увеличения эффективности модели стоит изменить архитектуру.

